# Problem 1

A training set is a fraction of the available data used create/train a machine learning model. A test set is a fraction of the available data that is reserved to test the model performance after it has been created, this is necessary to prevent the model from "memorizing" the data set.

# Problem 2

It is necessary to reserve data in order to prevent memorizing the data and over fitting.

# Problem 3

Advantages: simple, easy to implement, no assumptions. Disadvantages: features must be on same scale, slow/suffers from large data sets and large number of features (must calculate the distance for all data points/features.

# Problem 4

Bias is the difference between the expected value of the model and the actual data. Variance is the expected difference between the value of the model for a given x and it's expected value, squared. Low complexity models are insensitive, highly biased, giving the similar results despite changes in the underlying data. High complexity models are super-sensitivt, highly variant, giving widely different results for differnt datasets.

# Problem 5

At low k, KNN suffers from high bias. At high K it suffers from high variance.

# Problem 6

When suffering high variance, reduce features or switch to a more biased model.

# Problem 7

When suffering from high bias, add more features or swtich to a less biased model.

# Problem 8

Cross validation

# Question 1

### Load in data and scale

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

prostate = pd.read_csv('https://raw.githubusercontent.com/galvin-mj/DAT_ATL_15/master/Datasets/ProstateCancer.csv')
prostate.head()
del prostate['Unnamed: 0']
prostate_train = prostate[prostate['train'] == 'T']
prostate_test = prostate[prostate['train'] == 'F'] 

from sklearn import preprocessing
x_train = preprocessing.scale(prostate_train.loc[:,'lcavol':'pgg45'])
y_train = preprocessing.scale(prostate_train.loc[:,'lpsa'])
x_test = preprocessing.scale(prostate_test.loc[:,'lcavol':'pgg45'])
y_test = preprocessing.scale(prostate_test.loc[:,'lpsa'])


### Linear regression

In [3]:
from sklearn import linear_model
model = linear_model.LinearRegression()

model.fit(x_train,y_train)

prediction = model.predict(x_test)

from sklearn import metrics
MSE = metrics.mean_squared_error(y_test,prediction)

print "MSE is", MSE

MSE is 0.491127418061


### KNN

In [6]:
from sklearn import grid_search
from sklearn import neighbors
param_grid = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10]}]
knn = neighbors.KNeighborsRegressor()
rgr = grid_search.GridSearchCV(knn, param_grid)

rgr.fit(x_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_neighbors=5, p=2, weights='uniform'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}],
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [7]:
rgr.best_params_

{'n_neighbors': 6}

In [8]:
knn2 = neighbors.KNeighborsRegressor(n_neighbors=6)

knn2.fit(x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_neighbors=6, p=2, weights='uniform')

In [9]:

prediction = knn2.predict(x_test)

In [11]:
from sklearn import metrics

MSE = metrics.mean_squared_error(y_test,prediction)
print "MSE is", MSE

MSE is 0.649347684238


### Ridge regression

In [15]:
param_grid = [{'alpha' : np.linspace(1e-8,1000,1000)}]
ridge_rgr = linear_model.Ridge()
model = grid_search.GridSearchCV(ridge_rgr, param_grid)
model.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid=[{'alpha': array([  1.00000e-08,   1.00100e+00, ...,   9.98999e+02,   1.00000e+03])}],
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [16]:
model.best_estimator_

Ridge(alpha=192.19219220027026, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, solver='auto', tol=0.001)

In [18]:
ridge_rgr2 = linear_model.Ridge(alpha = 192.19)
ridge_rgr2.fit(x_train, y_train)
prediction = ridge_rgr2.predict(x_test)
MSE = metrics.mean_squared_error(y_test,prediction)
print "MSE is", MSE

MSE is 0.603221437211


In [19]:
ridge_rgr2.coef_

array([ 0.14037975,  0.09756156,  0.02151208,  0.05816745,  0.09671267,
        0.06487368,  0.04172619,  0.06402935])

### Lasso Regression

In [20]:
param_grid = [{'alpha' : np.linspace(1e-8,100,1000)}]
lasso_rgr = linear_model.LassoCV()
lasso_rgr.fit(x_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [21]:
prediction = lasso_rgr.predict(x_test)

In [22]:
metrics.mean_squared_error(y_test, prediction)

0.44275503080999246

In [23]:
lasso_rgr.coef_

array([ 0.47933305,  0.20137619, -0.        ,  0.11393298,  0.16220515,
       -0.        ,  0.        ,  0.07144221])

# Question 2

In [5]:
prostate_test.loc[:,'lcavol':'pgg45'].corr()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
lcavol,1.000000,0.203455,0.096055,-0.076500,0.398673,0.636998,0.453406,0.309566
lweight,0.203455,1.000000,0.526248,0.495874,0.072752,0.205169,0.173297,0.257143
age,0.096055,0.526248,1.000000,0.543672,0.079422,0.049455,0.092624,0.230829
lbph,-0.076500,0.495874,0.543672,1.000000,0.043540,0.175657,0.170479,0.379522
svi,0.398673,0.072752,0.079422,0.043540,1.000000,0.683579,0.356348,0.394816
lcp,0.636998,0.205169,0.049455,0.175657,0.683579,1.000000,0.593699,0.582940
gleason,0.453406,0.173297,0.092624,0.170479,0.356348,0.593699,1.000000,0.782083
pgg45,0.309566,0.257143,0.230829,0.379522,0.394816,0.582940,0.782083,1.000000


Yes, the pgg45 and the gleason features are highly correlated (0.78). Also the lcp and svi were highly correlated (0.68)

# Question 3

In [24]:
lasso_rgr.coef_

array([ 0.47933305,  0.20137619, -0.        ,  0.11393298,  0.16220515,
       -0.        ,  0.        ,  0.07144221])

Looking at the lasso regression, the coeffcients that were most important were lcavol (0.47) and lweight (0.20). The coefficients that were the least important were age, gleason, and lcp.

# Question 4

Lasso worked the best with an MSE of 0.44

# Question 5

I had to scale the data in order to get the knn to work without having a huge MSE. Also, I could have removed the correlated coefficients from the regression to see if that would fit better.